<a href="https://colab.research.google.com/github/StoneMason495/cap-comp215/blob/main/215Project2ZM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Daisy World: Testing the Gaia Hypothesis
###By: Zach Mason, COMP 215

In [7]:
import numpy as np
import matplotlib as plt
import random
from dataclasses import dataclass

##Daisy Class:
I created a simple data class to create each daisy, which only has its colour (inputted as a string), and the albedo, or amount of light reflected by a surface in decimal/fraction form (inputted as a float)
I chose the albedo values of 0.25 for black and 0.75 for white, as [this website](https://serc.carleton.edu/NAGTWorkshops/complexsystems/activities/daisyworld.html) utilized those values as well. The 0.75 value means that the white daisies will reflect 75% of the light shone upon them, where as the black daisies (with an albedo of 0.25) will absorb 75% of the light, and therefore the heat

In [23]:
B = lambda T: 1 - 0.003265*(22.5-T)**2
@dataclass
class Daisy: #a dataclass, with only the colour of the daisy, and the albedo value (fractional value of light that a surface reflects)
  color : str
  albedo : float
  pop : float
  
  def growth_rate(self, temp):
    growth_rate = self.pop * B(temp)
    return growth_rate if growth_rate > 0 else 0
  def death(self, gamma=0.3):
    if self.pop > 0.01:
      return self.pop * gamma
    else:
      return 0
  def step(self):
    return 1

white = Daisy('white', 0.75, 0.001)
black = Daisy('black', 0.25, 0.001)

white.growth_rate(31), black.growth_rate(22.5)
white.death()

0.009

##Daisyworld Class:

In [6]:
class Daisyworld:
  def __init__(self, b_daisy, w_daisy, p=1.0):
    self.black = b_daisy
    self.white = w_daisy
  def step(self):
    return 1